请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 赛题介绍

图神经网络（Graph Neural Network）是一种专门处理图结构数据的神经网络，目前被广泛应用于推荐系统、金融风控、生物计算中。图神经网络的经典问题主要有三种，包括节点分类、连接预测和图分类三种。本次比赛是图神经网络7日打卡课程的大作业，主要让同学们熟悉如何图神经网络处理节点分类问题。

数据集为论文引用网络，图由大量的学术论文组成，节点之间的边是论文的引用关系，每一个节点提供简单的词向量组合的节点特征。我们的目的是给每篇论文推断出它的论文类别。

# 运行方式
依次运行下方的cell，完成后下载submission.csv提交结果即可;

# 代码整体逻辑
1. 最重要的是使用的新版本的paddle 和pgl; pgl使用参考github 的代码示例
paddle 版本 2.3.1
pgl 版本 2.2.3post0
2. 代码逻辑大同小异，一个主函数main，在主流程里:
- 2.1 调用load函数读取数据集，构建图，划分训练集、验证集、测试集
- 2.2 下面是模型训练的过程：定义模型及超参数，定义优化器、损失函数，执行300个epoch，每20轮输出训练集、验证集的loss和auc
- 2.3 对测试集的id进行预测并输出结果
3. 可以修改定义的模型及超参数

# 思路介绍及后续完善

1. 用最简单的gcn，代码框架参考baseline 及github中pgl的代码示例；
2. 考虑到我们的节点分类有35个，调整了隐藏层单元128，两层隐藏
3. 后续可以考虑调整gcn的隐藏单元等超参数；使用gat,unimp等模型

# 环境配置

In [3]:
import paddle

In [4]:
paddle.__version__

'2.3.1'

In [5]:
!pip install pgl==2.2.3post0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 40.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import paddle
import paddle.nn as nn
from pgl.utils.logger import log
from paddle.optimizer import Adam
import pgl
import paddle.fluid as fluid
import numpy as np
import time
import pandas as pd
from collections import namedtuple,Counter
import tqdm

# 具体代码分享

In [9]:
Dataset = namedtuple("Dataset", 
               ["graph", "num_classes", "train_index",
                "train_label", "valid_index", "valid_label", "test_index", "test_label"])

In [22]:
def load_edges(num_nodes, self_loop=True, add_inverse_edge=True):
    # 从数据中读取边
    edges = pd.read_csv("/home/aistudio/data/data61620/edges.csv", header=None, names=["src", "dst"]).values

    if add_inverse_edge:
        edges = np.vstack([edges, edges[:, ::-1]])

    if self_loop:
        src = np.arange(0, num_nodes)
        dst = np.arange(0, num_nodes)
        self_loop = np.vstack([src, dst]).T
        edges = np.vstack([edges, self_loop])
    
    return edges

def load():
    # 从数据中读取点特征和边，以及数据划分
    node_feat = np.load("/home/aistudio/data/data61620//feat.npy")
    num_nodes = node_feat.shape[0]
    edges = load_edges(num_nodes=num_nodes, self_loop=True, add_inverse_edge=True)
    graph = pgl.graph.Graph(num_nodes=num_nodes, edges=edges, node_feat={"feat": node_feat})
    
    indegree = graph.indegree()
    norm = np.maximum(indegree.astype("float32"), 1)
    norm = np.power(norm, -0.5)
    graph.node_feat["norm"] = np.expand_dims(norm, -1)
    
    # 训练集、验证集划分
    df = pd.read_csv("/home/aistudio/data/data61620/train.csv")
    node_index = df["nid"].values
    node_label = df["label"].values
    train_part = int(len(node_index) * 0.8)
    train_index = node_index[:train_part]
    train_label = node_label[:train_part]
    valid_index = node_index[train_part:]
    valid_label = node_label[train_part:]
    # 读取测试集
    test_index = pd.read_csv("/home/aistudio/data/data61620/test.csv")["nid"].values
    test_label = np.zeros((len(test_index), 1), dtype="int64")
    
    # 要转成tensor类型，并注意维度
    train_index = paddle.to_tensor(np.expand_dims(train_index, -1))
    valid_index = paddle.to_tensor(np.expand_dims(valid_index, -1))
    test_index = paddle.to_tensor(np.expand_dims(test_index, -1))
    
    train_label = paddle.to_tensor(np.expand_dims(train_label, -1))
    valid_label = paddle.to_tensor(np.expand_dims(valid_label, -1))
    test_label = paddle.to_tensor(test_label)
    
    dataset = Dataset(graph=graph, 
                    train_label=train_label,
                    train_index=train_index,
                    valid_index=valid_index,
                    valid_label=valid_label,
                    test_index=test_index,
                    test_label=test_label,
                      num_classes=35)
    return dataset

In [23]:
# 检查一下load结果
dataset = load()
print(dataset.graph.node_feat["feat"].shape)

(130644, 100)


In [24]:
dataset.test_index

Tensor(shape=[37311, 1], dtype=int64, place=Place(cpu), stop_gradient=True,
       [[118264],
        [411   ],
        [118364],
        ...,
        [117847],
        [117848],
        [117934]])

In [25]:
class GCN(nn.Layer):
    """Implement of GCN
    """

    def __init__(self,
                 input_size,
                 num_class,
                 num_layers=1,
                 hidden_size=64,
                 dropout=0.5):
        super(GCN, self).__init__()
        self.num_class = num_class
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.gcns = nn.LayerList()
        for i in range(self.num_layers):
            if i == 0:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        input_size,
                        self.hidden_size,
                        activation="relu",
                        norm=True))
            else:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        self.hidden_size,
                        self.hidden_size,
                        activation="relu",
                        norm=True))
            self.gcns.append(nn.Dropout(self.dropout))
        self.gcns.append(pgl.nn.GCNConv(self.hidden_size, self.num_class))

    def forward(self, graph, feature):
        for m in self.gcns:
            if isinstance(m, nn.Dropout):
                feature = m(feature)
            else:
                feature = m(graph, feature)
        return feature

In [19]:
class GAT(nn.Layer):
    """Implement of GAT
    """

    def __init__(
            self,
            input_size,
            num_class,
            num_layers=1,
            feat_drop=0.6,
            attn_drop=0.6,
            num_heads=8,
            hidden_size=8, ):
        super(GAT, self).__init__()
        self.num_class = num_class
        self.num_layers = num_layers
        self.feat_drop = feat_drop
        self.attn_drop = attn_drop
        self.num_heads = num_heads
        self.hidden_size = hidden_size
        self.gats = nn.LayerList()
        for i in range(self.num_layers):
            if i == 0:
                self.gats.append(
                    pgl.nn.GATConv(
                        input_size,
                        self.hidden_size,
                        self.feat_drop,
                        self.attn_drop,
                        self.num_heads,
                        activation='elu'))
            elif i == (self.num_layers - 1):
                self.gats.append(
                    pgl.nn.GATConv(
                        self.num_heads * self.hidden_size,
                        self.num_class,
                        self.feat_drop,
                        self.attn_drop,
                        1,
                        concat=False,
                        activation=None))
            else:
                self.gats.append(
                    pgl.nn.GATConv(
                        self.num_heads * self.hidden_size,
                        self.hidden_size,
                        self.feat_drop,
                        self.attn_drop,
                        self.num_heads,
                        activation='elu'))
    def forward(self, graph, feature):
        for m in self.gats:
            feature = m(graph, feature)
        return feature

In [26]:
# 模型训练 前向-loss-反向传播
def train(model, index, label, graph, criterion, optim):
    model.train()
    pred = model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, index)
    loss = criterion(pred, label)
    loss.backward()
    acc = paddle.metric.accuracy(input=pred, label=label, k=1)
    optim.step()
    optim.clear_grad()
    return loss, acc

# 用于评估，只有前向计算和loss/auc
@paddle.no_grad()
def eval_test(model, index, label, graph, criterion):
    model.eval()
    pred = model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, index)
    loss = criterion(pred, label)
    acc = paddle.metric.accuracy(input=pred, label=label, k=1)
    return loss, acc


def set_seed(seed):
    paddle.seed(seed)
    np.random.seed(seed)

In [30]:
def main():
    
    dataset = load()
    dataset.graph.tensor()
    graph = dataset.graph
    
    
    train_index = dataset.train_index
    train_label = dataset.train_label

    val_index = dataset.valid_index
    val_label = dataset.valid_label

    test_index = dataset.test_index
    test_label = dataset.test_label
    

    #return  dataset, graph
    criterion = paddle.nn.loss.CrossEntropyLoss()

    dur = []

    for run in range(1):
        cal_val_acc = []
        cal_test_acc = []
        cal_val_loss = []
        cal_test_loss = []
        gnn_model = GCN(input_size=graph.node_feat["feat"].shape[1],
                        num_class=dataset.num_classes,
                        num_layers=2,
                        dropout=0.5,
                        hidden_size=128)
#         gnn_model = GAT(input_size=graph.node_feat["feat"].shape[1],
#                         num_class=dataset.num_classes,
#                         num_layers=3,
#                         feat_drop=0.5,
#                         attn_drop=0.5,
#                         num_heads=8,
#                         hidden_size=8)
        optim = Adam(
            learning_rate=0.01,
            parameters=gnn_model.parameters(),
            weight_decay=0.0005)

        for epoch in tqdm.tqdm(range(1, 301)):
            if epoch >= 3:
                start = time.time()
            train_loss, train_acc = train(gnn_model, train_index, train_label,
                                          graph, criterion, optim)
            if epoch >= 3:
                end = time.time()
                dur.append(end - start)
            val_loss, val_acc = eval_test(gnn_model, val_index, val_label, graph,
                                     criterion)
            cal_val_acc.append(val_acc.numpy())
            cal_val_loss.append(val_loss.numpy())
            if epoch % 20 == 0:
                print("Epoch", epoch, "Train loss", np.array(train_loss)[0], "Valid loss", np.array(val_loss)[0])
                print("Train Acc", np.array(train_acc)[0], "Valid Acc", np.array(val_acc)[0])

    log.info("Average Speed %s sec/ epoch" % (np.mean(dur)))
    # 预测
    gnn_model.eval()
    pred = gnn_model(graph, graph.node_feat["feat"])
    pred = paddle.gather(pred, test_index)
    submission = pd.DataFrame(data={
                            "nid": np.array(test_index).reshape(-1),
                            "label": np.argmax(pred,1)
                        })
    
    return gnn_model, pred, submission

In [28]:
model, preds, submission = main() # 保留60个epoch结果 当初未保留模型

 33%|███▎      | 20/60 [02:35<05:13,  7.83s/it]

Epoch 20 Train loss 2.0737271 Valid loss 1.956179
Train Acc 0.4419271 Valid Acc 0.4672172


 67%|██████▋   | 40/60 [05:10<02:38,  7.95s/it]

Epoch 40 Train loss 1.5756513 Valid loss 1.4543743
Train Acc 0.56999713 Valid Acc 0.5942194


100%|██████████| 60/60 [07:46<00:00,  7.77s/it]
[INFO] 2022-08-05 00:25:38,164 [967635228.py:   61]:	Average Speed 5.036017306919756 sec/ epoch


Epoch 60 Train loss 1.3513439 Valid loss 1.2379695
Train Acc 0.6195985 Valid Acc 0.64305544


In [29]:
# 预测和提交
submission.to_csv("submission.csv", index=False)